In [4]:
import yfinance as yf

def extract_data_yf(tickers, Period = "1y"): # Note: Remember to make a way to delelte these files after use. Since they are only temporary files.
    data = {}
    for ticker in tickers:
        df = yf.download(ticker, period=Period, interval="1d",auto_adjust=True, progress=False)
        filename = f"temp_{ticker}_{Period}.csv"
        df.to_csv(filename)
        data[ticker] = df
    return data

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

df = extract_data_yf(["AAPL"], Period="max")

df = pd.DataFrame(data)
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", inplace=True)

# 2. Feature Engineering
df["dayofweek"] = df.index.dayofweek
df["day"] = df.index.day
df["month"] = df.index.month
df["is_month_end"] = df.index.is_month_end.astype(int)
df["is_month_start"] = df.index.is_month_start.astype(int)
df["Close_lag1"] = df["Close"].shift(1)
df["Volume_lag1"] = df["Volume"].shift(1)
df["Close_rolling_mean_3"] = df["Close"].rolling(3).mean()
df["Close_rolling_std_3"] = df["Close"].rolling(3).std()
df["target"] = df["Close"].shift(-1)
df.dropna(inplace=True)

# 3. Prepare Features and Target
features = [
    "Open", "High", "Low", "Volume", "dayofweek", "day", "month",
    "Close_lag1", "Volume_lag1", "Close_rolling_mean_3", "Close_rolling_std_3"
]
X = df[features].values
y = df["target"].values.reshape(-1, 1)

# 4. Normalize Features
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# 5. Reshape for LSTM [samples, timesteps, features]
X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# 6. Build LSTM Model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_lstm.shape[1], X_lstm.shape[2])),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# 7. Train
model.fit(X_lstm, y_scaled, epochs=100, verbose=0)

# 8. Save Model and Scalers
model.save("lstm_stock_model.keras")
joblib.dump(scaler_X, "scaler_X.joblib")
joblib.dump(scaler_y, "scaler_y.joblib")


ModuleNotFoundError: No module named 'tensorflow'